In [1]:
from llama_index.core import StorageContext, load_index_from_storage
from constants import embedding_model

storage_context = StorageContext.from_defaults(persist_dir="index/")
index = load_index_from_storage(storage_context, embedding_model=embedding_model)

In [2]:
from llama_index.core.tools import QueryEngineTool
from constants import llm_model

query_engine = index.as_query_engine(llm_model=llm_model, similarity_top_k=5)
rag_tool = QueryEngineTool.from_defaults(
    query_engine, 
    name="reesarch_paper_query_engine_tool", 
    description="A RAG engine with recent research papers."
    )


In [3]:
from IPython.display import display, Markdown

def display_prompt_dictionary(prompt_dictionary):
    for key, prompt in prompt_dictionary.items():
        display(Markdown(f"**Prompt key**:{key}"))
        print(prompt.get_template())

In [4]:
prompts_dictionary = query_engine.get_prompts()
display_prompt_dictionary(prompts_dictionary)

**Prompt key**:response_synthesizer:text_qa_template

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt key**:response_synthesizer:refine_template

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [5]:
from tools import download_pdf, fetch_from_arxiv
from llama_index.core.tools import FunctionTool

download_pdf_tool = FunctionTool.from_defaults(
    download_pdf, 
    name="download_pdf_file_tool", 
    description="Python function that downloads PDFs from provided URLs."
    )

fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_from_arxiv, 
    name="fetch_from_arxiv", 
    description="Download the {max_results} recent papers regarding the topic {title} from arXiv."
)

In [6]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [rag_tool, download_pdf_tool, fetch_arxiv_tool], 
    llm_model=llm_model,
    max_iterations=12,
    verbose=True,
     handle_parsing_errors=True,  # Helps recover from tool input/output issues
    return_intermediate_steps=True  # For debugging if needed
    )

In [7]:
query_template = """I am intersted in {topic}.
Find papers in your knowledge database related to this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to {topic}'
If there are not, could you fetch the recent one from arXiv?
IMOPORTANT: do NOT download the papers UNLESS the user asks for it.
"""

In [8]:
answer = agent.chat(query_template.format(topic="Large Language Models"))

> Running step bd1bb7c3-7ea1-4869-9c23-6f3e2df59563. Step input: I am intersted in Large Language Models.
Find papers in your knowledge database related to this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to Large Language Models'
If there are not, could you fetch the recent one from arXiv?
IMOPORTANT: do NOT download the papers UNLESS the user asks for it.

Thought: The user is interested in finding research papers related to Large Language Models. I can use the research_paper_query_engine_tool to search for papers in the knowledge database.
Action: reesarch_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, author and link to download the PDF for papers related to Large Language Models'}
Observation: Title: How do Large Language Models Understand Relevance? A Mechanistic Interpretability Perspective
Summary: Recent studies have shown that large la

In [9]:
Markdown(answer.response)

1. Title: How do Large Language Models Understand Relevance? A Mechanistic Interpretability Perspective
   Summary: The paper investigates how different Large Language Model (LLM) modules contribute to relevance judgment through mechanistic interpretability. It analyzes the roles of various model components and identifies a multi-stage process in generating relevance judgments.
   Authors: Qi Liu, Jiaxin Mao, Ji-Rong Wen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07898v1)

2. Title: Porting an LLM based Application from ChatGPT to an On-Premise Environment
   Summary: The paper discusses the porting process of a real-life application using ChatGPT from a public cloud to an on-premise environment to address privacy and security concerns related to data.
   Authors: Teemu Paloniemi, Manu Setälä, Tommi Mikkonen
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07907v1)

3. Title: Redefining Machine Translation on Social Network Services with Large Language Models
   Summary: This paper introduces RedTrans, a Large Language Model tailored for Social Network Services (SNS) translation, addressing challenges in translating culturally nuanced content on SNS.
   Authors: Hongcheng Guo, Fei Zhao, Shaosheng Cao, Xinze Lyu, Ziyan Liu, Yue Wang, Boyang Wang, Zhoujun Li, Chonggang Lu, Zhe Xu, Yao Hu
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07901v1)

In [10]:
answer = agent.chat("""Downlod the following papers:
 For each paper:
 1. Process for one paper at a time. 
 2. State which paper number you are processing out of total.
 3. Complete a full download cycle before moving to next paper.
 4. Explicitly state when moving to the next paper.
 5. Provide a final summary only after all papers have been downloaded.                  
.""")

> Running step 9e0d7035-9e4a-4a2a-a3e0-bd8c2afe7036. Step input: Downlod the following papers:
 For each paper:
 1. Process for one paper at a time. 
 2. State which paper number you are processing out of total.
 3. Complete a full download cycle before moving to next paper.
 4. Explicitly state when moving to the next paper.
 5. Provide a final summary only after all papers have been downloaded.                  
.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2504.07898v1', 'output_file_name': 'How_do_Large_Language_Models_Understand_Relevance.pdf'}
Observation: PDF saved as 'papers\How_do_Large_Language_Models_Understand_Relevance.pdf'.
> Running step 0ddaabc9-032d-4141-a905-1a6d1965d7fb. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: download_pdf_file_tool
Ac

In [11]:
Markdown(answer.response)

All three papers related to Large Language Models have been successfully downloaded. The titles of the papers are:
1. How do Large Language Models Understand Relevance?
2. Porting an LLM based Application from ChatGPT to an On-Premise Environment
3. Redefining Machine Translation on Social Network Services

If you need any further information or assistance, feel free to ask!

In [12]:
answer = agent.chat(query_template.format(topic="Quntum Computing"))

> Running step 9b722651-39ce-4f25-ac3a-2fcb10f5db0c. Step input: I am intersted in Quntum Computing.
Find papers in your knowledge database related to this topic.
Use the folling template to query research_paper_query_engine_tool tool: 
'Provide title, summary, author and link to download the PDF for papers related to Quntum Computing'
If there are not, could you fetch the recent one from arXiv?
IMOPORTANT: do NOT download the papers UNLESS the user asks for it.

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: reesarch_paper_query_engine_tool
Action Input: {'input': 'Provide title, summary, author and link to download the PDF for papers related to Quantum Computing'}
Observation: There are no papers related to Quantum Computing in the provided context information.
> Running step 31e6766a-0809-432e-95df-f1e16a403f5a. Step input: None
Thought: I cannot answer the question with the provided tools.
Answer: There are no pap

In [14]:
Markdown(answer.response)

There are no papers related to Quantum Computing in the current database. If you would like, I can fetch the most recent papers on Quantum Computing from arXiv.

In [15]:
answer = agent.chat("fetch papers related to quantum computing from arXiv")

> Running step e4d0f324-550c-46f3-b766-43b1bef370b4. Step input: fetch papers related to quantum computing from arXiv
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: fetch_from_arxiv
Action Input: {'title': 'Quantum Computing', 'paper_count': 5}
Observation: [{'title': 'Quantum Speed Limit in Driven-dissipative Systems', 'summary': 'Every quantum operation that takes a system from one state to another is\nknown to have bounds on operation time, due to Heisenberg uncertainty\nprinciple. In open quantum systems (OQS), such bounds have been principally\naffected by system environment coupling. In the recent past, drives on OQS have\nshown to give rise to drive-induced dissipation (DID). In this work, we\ninvestigate how DID affects the quantum speed limits. To this end, we use a\nrecently-reported quantum master equation that takes into account environment\nfluctuations and provide a closed form estimate of drive-induced 

In [16]:
Markdown(answer.response)

1. Title: Quantum Speed Limit in Driven-dissipative Systems
   Summary: The paper investigates the impact of drive-induced dissipation on quantum speed limits in open quantum systems. It provides insights into optimizing quantum control in open systems, enhancing fidelity and efficiency for practical quantum computing under dissipative conditions.
   Authors: Sarfraj Fency, Riddhi Chatterjee, Rangeet Bhattacharyya
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07931v1)

2. Title: Temporal Tensors and Quantum Shortcut Dynamics in a Supermaze of Multidimensional Time
   Summary: This paper introduces a theoretical framework combining multiple time dimensions, quantum shortcut dynamics, and complex topological structures to explore novel phenomena in quantum and classical systems. It discusses implications for quantum computing and connections to mathematical paradoxes and physical principles.
   Author: Koffka Khan
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07900v1)

3. Title: QubitHammer Attacks: Qubit Flipping Attacks in Multi-tenant Superconducting Quantum Computers
   Summary: The paper presents QubitHammer attacks targeting superconducting quantum computers in multi-tenant cloud environments. It demonstrates the potential security vulnerabilities in shared quantum systems and the effectiveness of these attacks in degrading victim computation.
   Authors: Yizhuo Tan, Navnil Choudhury, Kanad Basu, Jakub Szefer
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07875v1)

4. Title: A quantum computing approach to beam angle optimization
   Summary: This study introduces QC-BAO, a hybrid quantum-classical approach for beam angle optimization in radiation therapy treatment planning. It leverages quantum computing to enhance treatment plan quality, demonstrating improved conformity index for target coverage and reduced doses to organs-at-risk.
   Authors: Nimita Shinde, Ya-Nan Zhu, Haozheng Shen, Hao Gao
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07844v1)

5. Title: Efficient Formal Verification of Quantum Error Correcting Programs
   Summary: The paper proposes an efficient verification framework for Quantum Error Correcting (QEC) programs, essential for fault-tolerant quantum computation. It introduces an assertion logic and program logic tailored for QEC programs, along with an automated QEC verifier for various fault-tolerant scenarios.
   Authors: Qifan Huang, Li Zhou, Wang Fang, Mengyu Zhao, Mingsheng Ying
   PDF URL: [Download PDF](http://arxiv.org/pdf/2504.07732v1)